# Dataset Cleaning and Exploration

To prepare for our ultimate task of using molecular embeddings to predict abundances and detectability, we need to put our dataset into working order. This is typically referred to as "cleaning", where we're making sure that the data will be valid (free of missing data, for example) and duplicate entries are removed. We will also need to inspect the data to make sure that entries we expect to be there are present, as well as observe some general trends where we can. 

Since we're looking at such a large dataset, we need to be able to inspect it from a microscopic and a macroscopic level. Combining both perspectives gives you an overview of what the dataset looks like, and in turn may give you insight into why/how a machine learning model behaves the way it does.

The first part of this notebook will be combining the three datasets: QM9, ZINC15, and KIDA. The latter is actually obtained by scraping their website, i.e. extracting the relevant information from tables in websites. 

In [1]:
from pathlib import Path
from tempfile import NamedTemporaryFile
import fileinput
import os

import numpy as np
import pandas as pd
from mol2vec import features
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from gensim.models import word2vec

Traceback (most recent call last):
  File "/home/kelvin/anaconda3/envs/rdkit/lib/python3.6/site-packages/rdkit/Chem/PandasTools.py", line 130, in <module>
    if 'display.width' in pd.core.config._registered_options:
AttributeError: module 'pandas.core' has no attribute 'config'


## Loading and combining SMILES from the two datasets

In [2]:
qm9 = pd.read_csv("../../data/external/gdb9_prop_smiles.csv.tar.gz")
smi_list = qm9["smiles"].dropna().to_list()

In [3]:
for smi_file in Path("../../data/external").rglob("*/*.smi"):
    temp = smi_file.read_text().split("\n")[1:]
    for line in temp:
        if len(line) != 0:
            smi_list.append(line.split(" ")[0])

In [4]:
print(f"Number of molecules: {len(smi_list)}")

Number of molecules: 1552425


## Extracting SMILES from KIDA

Since our database contains mostly terrestrial/stable molecules, we need to augment this set with astronomically relevant molecules. KIDA is one of the biggest reaction networks used in astrochemistry, and is therefore a nice collection molecules that may or may not be found in space (at least of interest).

To perform this, we'll scrape the KIDA website below.

In [5]:
import requests
import datetime
from bs4 import BeautifulSoup

In [6]:
url = requests.get("http://kida.astrophy.u-bordeaux.fr/species.html")

In [7]:
print(f"""Last retrieved: {url.headers["Date"]}""")

Last retrieved: Thu, 29 Oct 2020 15:18:11 GMT


In [8]:
RERUN = False

In [9]:
if RERUN:
    date = url.headers["Date"]
    # cut the date, replace spaces with underscores for naming
    date = date[5:16].replace(" ", "_")
    # save the webpage for reference. If KIDA decides to go bottom up we
    # will always have a copy of this data
    with open(f"../data/kida-site_{date}.html", "w+") as write_file:
        write_file.write(str(url.content))
    soup = BeautifulSoup(url.content)
    # the first and only table on the page corresponds to the molecules
    molecule_table = soup.find_all("table", "table")[0]

In [10]:
if RERUN:
    map_dict = dict()
    for row in soup.find_all("tr"):
        # some InChI are missing, this sets a default value
        inchi = None
        for index, column in enumerate(row.find_all("td")):
            # loop over columns in each row, and grab the second and
            # third columns which are formulae and InChI
            if index == 1:
                # strip twice because the first header is parsed funnily
                name = column.text.strip().strip()
            if index == 2:
                inchi = column.text.strip()
        map_dict[name] = inchi
    # Just for reference, dump the KIDA mapping as a dataframe
    kida_df = pd.DataFrame.from_dict(map_dict, orient="index").reset_index()
    kida_df.columns = ["Formula", "InChI"]
    kida_df.to_csv(f"../data/kida-molecules_{date}.csv", index=False)
else:
    kida_df = pd.read_csv("../../data/external/kida-molecules_05_Jul_2020.csv")

In [11]:
def inchi_to_smiles(inchi: str):
    inchi = str(inchi)
    if len(inchi) != 0:
        mol = Chem.MolFromInchi(inchi, sanitize=False, removeHs=False)
        if mol:
            smiles = Chem.MolToSmiles(mol, canonical=True)
            return smiles
    else:
        return ""

Now we convert all the InChI codes from KIDA into SMILES through RDKit. Initially I was most worried about this because KIDA has strange molecules, and as we see below RDKit has plenty to complain about. The attitude we're taking here is to ignore the ones that don't play by the rules, and we'll worry about them some other time.

In [12]:
# This applies our filtering function we defined above
kida_df["SMILES"] = kida_df["InChI"].apply(inchi_to_smiles)

RDKit WARNING: [11:18:35] expect radical to be either 2 or 3 while getting . Ignore radical.
RDKit WARNING: [11:18:35] expect radical to be either 2 or 3 while getting . Ignore radical.
RDKit WARNING: [11:18:35] WARNING:  Problems/mismatches: Mobile-H( Charge(s): Do not match)
RDKit WARNING: [11:18:35] WARNING:  Problems/mismatches: Mobile-H( Charge(s): Do not match)
RDKit WARNING: [11:18:35] WARNING:  Problems/mismatches: Mobile-H( Charge(s): Do not match)
RDKit WARNING: [11:18:35] WARNING:  Problems/mismatches: Mobile-H( Charge(s): Do not match)
RDKit WARNING: [11:18:35] WARNING:  Problems/mismatches: Mobile-H( Charge(s): Do not match)
RDKit WARNING: [11:18:35] WARNING:  Problems/mismatches: Mobile-H( Charge(s): Do not match)
RDKit WARNING: [11:18:35] WARNING:  Problems/mismatches: Mobile-H( Charge(s): Do not match)
RDKit WARNING: [11:18:35] WARNING:  Problems/mismatches: Mobile-H( Charge(s): Do not match)
RDKit WARNING: [11:18:35] WARNING:  Problems/mismatches: Mobile-H( Charge(s)

In [13]:
# Extract only those with SMILES strings
kida_smiles = kida_df.loc[(kida_df["SMILES"].str.len() != 0.)].dropna()

In [15]:
# append all the KIDA entries to our full list
smi_list.extend(kida_smiles["SMILES"].to_list())

In [16]:
print(f"Number of molecules with KIDA: {len(smi_list)}")

Number of molecules with KIDA: 1553003


## Adding extra SMILES from TMC-1 inventory

Turns out we're missing some molecules (no surprise) that are known in TMC-1, but not inlcuded in our list. The code below will take data directly from the Google Sheets Jacqueline's set up.

In [17]:
tmc1 = pd.read_csv("../../data/raw/TMC-1_inventory.csv")

In [18]:
tmc1 = tmc1.loc[(tmc1["Isotopologue"] == 0)]

In [19]:
missing = tmc1.loc[~tmc1["SMILES"].isin(smi_list), "SMILES"].to_list()

In [20]:
smi_list.extend(missing)

In [23]:
print(f"Number of molecules with missing TMC-1: {len(smi_list)}")

Number of molecules with missing TMC-1: 1553054


## Adding some PAHs explicitly

In [24]:
with open("../../data/external/hand_pah.smi") as read_file:
    pah_smi = read_file.read().split()

In [25]:
smi_list.extend(pah_smi)

## Microscopic inspection

In this section, we're going to put certain aspects of the dataset under the microscope: for example, we want to check that certain molecules are contained in the set. Here, we'll be using our chemical intuition; the idea is to pick out a few molecules, and check if: (a) they are contained in our list, and (b) what their most similar molecules are.

In [27]:
def canonicize_smi(smi: str):
    """
    Function to convert any SMILES string into its canonical counterpart.
    This ensures that all comparisons made subsequently are made with the
    same SMILES representation, if it exists.
    """
    return Chem.MolToSmiles(Chem.MolFromSmiles(smi, sanitize=False), canonical=True)

In [28]:
smi_list = [canonicize_smi(smi) for smi in smi_list]

RDKit WARNING: [11:20:02] Conflicting single bond directions around double bond at index 1.
RDKit WARNING: [11:20:02]   BondStereo set to STEREONONE and single bond directions set to NONE.
RDKit WARNING: [11:20:02] Conflicting single bond directions around double bond at index 1.
RDKit WARNING: [11:20:02]   BondStereo set to STEREONONE and single bond directions set to NONE.
RDKit WARNING: [11:20:03] Conflicting single bond directions around double bond at index 1.
RDKit WARNING: [11:20:03]   BondStereo set to STEREONONE and single bond directions set to NONE.
RDKit WARNING: [11:20:03] Conflicting single bond directions around double bond at index 1.
RDKit WARNING: [11:20:03]   BondStereo set to STEREONONE and single bond directions set to NONE.
RDKit WARNING: [11:20:03] Conflicting single bond directions around double bond at index 1.
RDKit WARNING: [11:20:03]   BondStereo set to STEREONONE and single bond directions set to NONE.
RDKit WARNING: [11:20:03] Conflicting single bond direc

In [29]:
mol_df = pd.DataFrame(data=smi_list)
mol_df.columns = ["Raw"]

In [30]:
checklist = [
    "CC=O",             # acetaldehyde
    "c1ccccc1",         # benzene
    "c1ccc(cc1)C#N",    # benzonitrile
    "N#CC=C",           # vinyl cyanide
    "CC#N",             # methyl cyanide
    "C#CC#CC#N",
    "C#N",
    "C#CC#CC#CC#N",
    "C#CC#CC#CC#CC#N",
]

checklist = [canonicize_smi(smi) for smi in checklist]

In [31]:
mol_df.loc[:, "Check"] = mol_df["Raw"].isin(checklist)

In [32]:
# Make sure all the molecules we want are in the list
len(mol_df.loc[mol_df["Check"] == True]) == len(checklist)

False

In [35]:
molecular_weights = list()
for smi in smi_list:
    mol = Chem.MolFromSmiles(smi, sanitize=False)
    mol.UpdatePropertyCache(strict=False)
    molecular_weights.append(Descriptors.ExactMolWt(mol))

## Calculate descriptors

In [36]:
mol_df["MW"] = molecular_weights

## Drop duplicate entries, and save the data to disk

Our dataset actually contains a lot of duplicate entries. This step removes them, which would otherwise just waste our computation time.

In [37]:
mol_df

,Raw,Check,MW
0,C,False,16.031300
1,N,False,17.026549
2,O,False,18.010565
3,C#C,False,26.015650
4,C#N,True,27.010899
...,...,...,...
1553070,Nc1ccccc1,False,93.057849
1553071,c1ccncc1,False,79.042199
1553072,N#Cc1ccccc1,True,103.042199
1553073,c1ccc(nc1)-c1ccccn1,False,156.068748


In [38]:
mol_df.drop_duplicates("Raw", inplace=True)

In [42]:
mol_df.reset_index(inplace=True, drop=True)

In [46]:
mol_df.to_pickle("../../data/processed/combined_smiles.pkl.bz2")

## Tasks for data exploration

### Distribution of molecular weight

Plot a histogram of the molecular weight in our dataset.

### Counting functional group examples

In [96]:
# For example, number of carbonyls
mol_df["Raw"].str.contains("C=O").sum()

17782

## Dumping the SMILES data for mol2vec use

This only takes the SMILES column, and dumps it into a list of SMILES formatted and ready for `mol2vec` usage. Every SMILES is separated by a new line, and we don't include a header.

In [47]:
mol_df["Raw"].to_csv("../../data/interim/collected_smiles.smi", sep="\n", index=False, header=None)